# Setup

In [1]:
import importlib, os
from multiprocessing import Pool
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scripts import files, seqVis, processData
importlib.reload(seqVis)
importlib.reload(files)
importlib.reload(processData)

<module 'scripts.processData' from '/home/users/bentyeh/projects/disprot/scripts/processData.py'>

In [2]:
paths = files.getPaths()
os.chdir(paths['dirProject'])
nThreads = len(os.sched_getaffinity(0))
print('Current working directory: ' + os.getcwd())
print('Number of available CPUs: {}'.format(nThreads))

Current working directory: /home/users/bentyeh/projects/disprot
Number of available CPUs: 32


# Read data

Reference files

In [3]:
idMap = pd.read_table(paths['idMap'])
idMap.drop(idMap.index[idMap['d2p2_id'].isnull()], axis='index', inplace=True)
idMap.drop_duplicates(subset='d2p2_id', keep='first', inplace=True)
idMap.reset_index(drop=True, inplace=True)
idMap = idMap.astype({'d2p2_id': np.int64})
proteome = pd.read_table(paths['ensemblProteome63']).rename({'id': 'ensembl_peptide_id'}, axis='columns')
disorderDB = pd.read_table(paths['d2p2DisorderHuman_vsl2b'])

Gene lists

In [ ]:
gl_medComplex = pd.read_table(paths['gl_medComplex'])
gl_mediatorTFs_human = pd.read_table(paths['gl_mediatorTFs_human'])
gl_GTFs = pd.read_table(paths['gl_GTFs'])
gl_POLR = pd.read_table(paths['gl_POLR'])
gl_interest = pd.read_table(paths['gl_interest'])
toPlot = [gl_medComplex, gl_GTFs, gl_interest] # gl_POLR

# Plots

In [4]:
def plotSeqAndPropsHelper(seq, disorder, name):
    try:
        fig, axs = seqVis.plotSeqAndProps(seq, disorderDB, name=name)
        fig.savefig(os.path.join(paths['dirResults_seqPlots'], name + '.png'), dpi=300)
        plt.close(fig)
    except Exception as err:
        print(err)

def plotSeqAndPropsFromIDHelper(figPath, id, idType, idMap, disorderDB, proteome=None, name_col=None):
    # if not os.path.exists(figPath) or os.path.getmtime(figPath) < 1544688506 or os.path.getmtime(figPath) > 1544689480:
    try:
        # pdb.set_trace()
        fig, axs = seqVis.plotSeqAndPropsFromID(id, idType, idMap, disorderDB, proteome, name_col)
        fig.savefig(figPath, dpi=300)
        plt.close('all')
        return 1
    except Exception as err:
        if name_col is not None:
            name = idMap.loc[idMap[idType] == id, name_col].values.item()
        else:
            name = ''
        print('Error occurred for protein {} {} ({}): '.format(idType, id, name) + str(err))
        with open(paths['seqPlots_error'], mode='a') as f:
            print('Error occurred for protein {} {} ({}): '.format(idType, id, name) + str(err), file=f, flush=True)
        return 0

### All human genes

Multithreaded

In [ ]:
allIDs = set(proteome['ensembl_peptide_id'])
num_plots_created = 0
for i in range(idMap.shape[0]):
    figPath = paths['seqPlots'].format(idMap['hgnc_symbol'][i])
    if idMap['ensembl_peptide_id'][i] not in allIDs:
        # Protein ID is not in Ensembl GRCh37_63 (D2P2 ID probably matched via UniProt ID instead)
        # --> we assume that the D2P2 ID and corresponding disorder prediction are valid for the UniProt sequence
        # --> plot sequence from UniProt (idMap)
        # print('{} ({}) missing from proteome'.format(idMap['ensembl_peptide_id'][i], idMap['hgnc_symbol'][i]))
        num_plots_created += plotSeqAndPropsFromIDHelper(figPath, idMap['ensembl_peptide_id'][i], 'ensembl_peptide_id', idMap, disorderDB, idMap, 'hgnc_symbol')
    else:
        # Protein ID found in Ensembl GRCh37_63
        # --> plot sequence from Ensembl GRCh37_63 (proteome)
        num_plots_created += plotSeqAndPropsFromIDHelper(figPath, idMap['ensembl_peptide_id'][i], 'ensembl_peptide_id', idMap, disorderDB, proteome, 'hgnc_symbol')

Single threaded

In [ ]:
pool = Pool(nThreads)
allIDs = set(proteome['ensembl_peptide_id'])
for i in range(idMap.shape[0]):
    figPath = paths['seqPlots'].format(idMap['hgnc_symbol'][i])
    if idMap['ensembl_peptide_id'][i] not in allIDs:
        pool.apply_async(plotSeqAndPropsFromIDHelper,
                         (figPath, idMap['ensembl_peptide_id'][i], 'ensembl_peptide_id', idMap, disorderDB, idMap, 'hgnc_symbol'))
    else:
        pool.apply_async(plotSeqAndPropsFromIDHelper,
                         (figPath, idMap['ensembl_peptide_id'][i], 'ensembl_peptide_id', idMap, disorderDB, proteome, 'hgnc_symbol'))
pool.close()
pool.join()

/home/users/bentyeh/miniconda3/envs/rpy3_tmp/lib/python3.7/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '


Error occurred for protein ensembl_peptide_id ENSP00000419111 (IGKJ1): One or more characters in `seq` is not in `charOrder`.


/home/users/bentyeh/miniconda3/envs/rpy3_tmp/lib/python3.7/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '


Error occurred for protein ensembl_peptide_id ENSP00000485238 (FER1L5): Disorder region specified past sequence length
Error occurred for protein ensembl_peptide_id ENSP00000332963 (IZUMO1R): One or more characters in `seq` is not in `charOrder`.


/home/users/bentyeh/miniconda3/envs/rpy3_tmp/lib/python3.7/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '


Error occurred for protein ensembl_peptide_id ENSP00000491070 (C17orf100): Disorder region specified past sequence length
Error occurred for protein ensembl_peptide_id ENSP00000480376 (GOLGA6L6): Disorder region specified past sequence length
Error occurred for protein ensembl_peptide_id ENSP00000342790 (FAM25A): One or more characters in `seq` is not in `charOrder`.


/home/users/bentyeh/miniconda3/envs/rpy3_tmp/lib/python3.7/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '


Error occurred for protein ensembl_peptide_id ENSP00000494359 (UPK3BL2): Disorder region specified past sequence length
Error occurred for protein ensembl_peptide_id ENSP00000409542 (CCDC188): One or more characters in `seq` is not in `charOrder`.


/home/users/bentyeh/miniconda3/envs/rpy3_tmp/lib/python3.7/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '
/home/users/bentyeh/miniconda3/envs/rpy3_tmp/lib/python3.7/site-packages/matplotlib/tight_layout.py:176: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  warnings.warn('Tight layout not applied. The left and right margins '


Error occurred for protein ensembl_peptide_id ENSP00000399324 (FGF16): One or more characters in `seq` is not in `charOrder`.
Error occurred for protein ensembl_peptide_id ENSP00000489635 (DCX): Disorder region specified past sequence length
Error occurred for protein ensembl_peptide_id ENSP00000328729 (SELENOF): One or more characters in `seq` is not in `charOrder`.


### Transcription factors reported to interact with mediator subunits
Boija, A. et al. Transcription Factors Activate Genes through the Phase-Separation Capacity of Their Activation Domains. *Cell* 175, 1–14 (2018).

In [ ]:
uniprotProteome = pd.read_table(paths['uniprotProteome'])
d2p2IdMap = pd.read_table(paths['d2p2IdMap_raw'], header=None, names=['d2p2_id', 'uniprot_id'])
ids = processData.readFile(paths['gl_mediatorTFs'])
proteome = processData.generateProteome(ids, ref=uniprotProteome, idCol='id', nThreads=2)

In [ ]:
pool = Pool(nThreads)
print("Using {:d} threads...".format(pool._processes))
for i in range(proteome.shape[0]):
    uniprot_id = proteome.iloc[i, proteome.columns == 'id'].values.item()
    seq = proteome.iloc[i, proteome.columns == 'seq'].values.item()
    
    if uniprot_id not in set(d2p2IdMap['uniprot_id']):
        print('Protein {} not present in D2P2 database. Skipping.'.format(uniprot_id))
        continue
    
    d2p2_id = d2p2IdMap.loc[d2p2IdMap['uniprot_id'] == uniprot_id, 'd2p2_id'].values.item()
    
    if d2p2_id not in set(disorderDB['d2p2_id']):
        print('D2P2 ID {} not present in disorder database. Skipping.'.format(d2p2_id))
        continue

    disorder = disorderDB.loc[disorderDB['d2p2_id'] == d2p2_id, ['start', 'end']] \
                         .sort_values('start').to_records(index=False).tolist()
    name = proteome.iloc[i, proteome.columns == 'gn'].values.item()
    
    pool.apply_async(plotSeqAndPropsHelper, (seq, disorder, name))
pool.close()
pool.join()

### Gene lists of interest

In [ ]:
pool = Pool(nThreads)
for gl in toPlot:
    for hgnc_symbol in gl['hgnc_symbol']:
        figPath = paths['seqPlots'].format(hgnc_symbol)
        pool.apply_async(plotSeqAndPropsFromIDHelper,
                         (figPath, hgnc_symbol, 'hgnc_symbol', idMap, disorderDB, proteome))
pool.close()
pool.join()
#         if os.path.exists(figPath):
#             print('{} already exists. Skipping.'.format(figPath))
#             continue
#         try:
#             fig, axs = seqVis.plotSeqAndPropsFromID(hgnc_symbol, 'hgnc_symbol', idMap, d2p2DisorderHuman_vlxt)
#             fig.savefig(figPath, dpi=300)
#             plt.close(fig)
#         except Exception as err:
#             print(err)
#             continue

### Single genes of interest

In [ ]:
len(idMap.loc[idMap['hgnc_symbol'] == 'MUC16','seq'].values.item())

In [ ]:
seqVis.plotSeqAndPropsFromID('ENSP00000381008', 'ensembl_peptide_id', idMap, disorderDB, proteome)

In [ ]:
seqVis.plotSeqAndPropsFromID('FUS', 'hgnc_symbol', idMap, disorderDB, proteome)